---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [57]:
import pandas as pd
def answer_one():
    
    #Energy
    colnames = ['Country',
               'Energy Supply',
               'Energy Supply per Capita',
               '% Renewable']
    energy = pd.read_excel('Energy Indicators.xls',
                           skiprows=18,
                           skipfooter=38,
                           header=None,
                           usecols=[i for i in range(2,6)],
                           na_values = '...',
                           names = colnames)
    # Peta -> Giga
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    
    # Retirando numeros dos nomes
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    
    # Retirando parentes
    energy['Country'] = (energy['Country']
                         .str.replace(r"\(.*\)","")
                         .str.strip())
    
    # Renomear
    rename = {"Republic of Korea": "South Korea",
              "United States of America": "United States",
              "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
              "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy['Country'] = energy['Country'].replace(rename)
                            
    
    #GDP
    GDP = (pd.read_csv('world_bank.csv',
                      skiprows=5,
                      header=None)
                      .drop([i for i in range(1,50)], axis = 'columns'))
    
    # Rename col
    colnames = ['Country']+[str(i) for i in range(2006,2016)]
    GDP = GDP.rename(columns={x:y for x,y in zip(GDP.columns,colnames)})

    # Renomear
    rename = {"Korea, Rep.": "South Korea",
              "Iran, Islamic Rep.": "Iran",
              "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country'] = GDP['Country'].replace(rename)
    
    # ScimEn
    ScimEn = pd.read_excel('scimagojr-3.xlsx',
                           header=0)
    ScimEn = ScimEn.drop([i for i in range(15,len(ScimEn.index))], axis = 'index')
    # Merge
    ScimEn = pd.merge( ScimEn, energy, how='inner', left_on='Country', right_on='Country')
    ScimEn = pd.merge( ScimEn, GDP, how='inner', left_on='Country', right_on='Country')
    
    # Index
    ScimEn = ScimEn.set_index(['Country'])
    return ScimEn

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [58]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [59]:
import pandas as pd
def answer_two():
    #Energy
    colnames = ['Country',
               'Energy Supply',
               'Energy Supply per Capita',
               '% Renewable']
    energy = pd.read_excel('Energy Indicators.xls',
                           skiprows=18,
                           skipfooter=38,
                           header=None,
                           usecols=[i for i in range(2,6)],
                           na_values = '...',
                           names = colnames)
    # Peta -> Giga
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    
    # Retirando numeros dos nomes
    energy['Country'] = energy['Country'].str.replace('\d+', '')
    
    # Retirando parentes
    energy['Country'] = (energy['Country']
                         .str.replace(r"\(.*\)","")
                         .str.strip())
    
    # Renomear
    rename = {"Republic of Korea": "South Korea",
              "United States of America": "United States",
              "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
              "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy['Country'] = energy['Country'].replace(rename)
                            
    
    #GDP
    GDP = (pd.read_csv('world_bank.csv',
                      skiprows=5,
                      header=None)
                      .drop([i for i in range(1,50)], axis = 'columns'))
    
    # Rename col
    colnames = ['Country']+[str(i) for i in range(2006,2016)]
    GDP = GDP.rename(columns={x:y for x,y in zip(GDP.columns,colnames)})
    
    # Renomear
    rename = {"Korea, Rep.": "South Korea",
              "Iran, Islamic Rep.": "Iran",
              "Hong Kong SAR, China": "Hong Kong"}
    GDP['Country'] = GDP['Country'].replace(rename)
    
    # ScimEn
    ScimEn = pd.read_excel('scimagojr-3.xlsx',
                           header=0)
    #ScimEn = ScimEn.drop([i for i in range(15,len(ScimEn.index))], axis = 'index')
    # Merge
    Inner = pd.merge( ScimEn, energy, how='inner', left_on='Country', right_on='Country')
    Inner = pd.merge( Inner, GDP, how='inner', left_on='Country', right_on='Country')
    
    Outer = pd.merge( ScimEn, energy, how='outer', left_on='Country', right_on='Country')
    Outer = pd.merge( Outer, GDP, how='outer', left_on='Country', right_on='Country')
    
    return Outer.shape[0]-Inner.shape[0]

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [60]:
def answer_three():
    
    # Utiliza a resposta da primeira questão para produzir o dataframe onde serão feitas as estatisticas
    Top15 = answer_one()
    
    # Produz uma lista de colunas para serem avaliadas
    colnames = [str(i) for i in range(2006,2016)]
    
    # Produz a serie a partir das colunas 'colnames'
    avgGDP = (Top15[colnames]
              
              # Obtem a média para os 10 anos desconsideram valores NaN
              .apply(lambda x: np.nanmean(x[colnames]), axis=1)
              
              # Organiza os valores de maneira decrescente
              .sort_values(ascending = False))
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [61]:
import numpy as np

def answer_four():
    Top15 = answer_one()
    
    # Utiliza a resposta da questão tres para obter o key do pais com o sexto maior GDP
    sixth_key = ((answer_three()).keys()[5])
    
    # Faz a diferença dos valores de GDP de 2006 e 2015 utilizando o key obtido
    return ((Top15[['2006','2015']]).loc[sixth_key].diff())[1]

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [62]:
def answer_five():
    Top15 = answer_one()
    
    # retorna a média do valor  de Energia
    return np.mean(Top15['Energy Supply per Capita'])

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [63]:
def answer_six():
    Top15 = answer_one()
    
    # (Usa o metodo idxmax() para retornar o key associado ao valor maximo,
    #  Usa o metodo max() para retonar o maior valor de '% Renewable')
    maxRenew = (Top15['% Renewable'].idxmax(),Top15['% Renewable'].max())
    return maxRenew

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [64]:
import numpy as np

def answer_seven():
    
    # Obtem o df com os field's 'Citations' e 'Self-Citations'
    Top15 = answer_one()
    colnames= ['Citations','Self-citations']
    
    # Produz uma nova coluna com a razão de 'Self-Citations' por 'Citations'
    Top15['CitRatio'] = np.divide(Top15[colnames[1]],Top15[colnames[0]])
    
    # ( (Usa o metodo idxmax() para retornar o key associado ao valor maximo,
    #  Usa o metodo max() para retonar o maior valor de 'CitRatio')
    return (Top15['CitRatio'].idxmax(),Top15['CitRatio'].max())

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [65]:
def answer_eight():
    
    # Obtem o df com os field's presentes na lista colnames
    Top15 = answer_one()
    colnames= ['Energy Supply','Energy Supply per Capita']
    
    # Produz a coluna de estimativa de população
    Top15['PopEst'] = np.divide(Top15[colnames[0]],Top15[colnames[1]])
    
    # Organiza a serie com o metodo sort e exibe a terceira entrada
    return (Top15['PopEst']
            .sort_values(ascending=False)
            .keys()[2])

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [66]:
def answer_nine():
    Top15 = answer_one()
    
    # Produz coluna de estimativa da população
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    # Produza a estimativa de documentos citaveis por per capita
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    #plot9()
    
    # aplica o metodo corr para obter a correlação utilizando o metodo de pearson
    return Top15[['Energy Supply per Capita','Citable docs per Capita']].corr(method='pearson').iloc[0,1]

In [67]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [68]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [69]:
def answer_ten():
    Top15 = answer_one()
    
    # Produz a nova coluna 'HighRenew' utilizando o condicional para cada entrada e os metodos median para obter 
    # mediana e astype para inserir o booleano com int na coluna
    Top15['HighRenew'] = ((Top15['% Renewable']>=Top15['% Renewable']
                          .median())
                          .astype(int))
    return Top15['HighRenew']

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [70]:
import numpy as np
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    # Cria a estimativa da população
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    # Utiliza o dicionario para agrupar por continente com o metodo groupby e o 
    # metodo agg para produzir as novas colunas
    Continents = Top15.groupby(ContinentDict)['PopEst'].agg({'size': np.size,
                                                             'sum':np.sum,
                                                             'mean':np.mean,
                                                             'std':np.std})
    return Continents

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [71]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    # utiliza o metodo cut para dividir os bins e o metodo count para fazer a contagem
    Cont  = Top15.groupby([ContinentDict,pd.cut(Top15['% Renewable'],5)]).count()#['% Renewable'].agg({'bin':np.size})
    return Cont['Rank'].dropna()

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [76]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    # Utiliza o metodo map para reformatar as entradas da serie
    Resp = Top15['PopEst'].map('{:,}'.format)
    return Resp

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [73]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!